`04_memory`

# Memory (대화 내용 기억)

- 대화 내용 기억
- LLM은 기본적으로 대화 내용을 기억하지 않는다(stateless)
- 이전 대화내용을 계속 프롬프트에 주입해야 함

1. Short-Term Memory
    - 단기기억: 한 대화 세선에 대한 기억
2. Long-Term Memory
    - 장기기억: 전체 세션에서 추출한 중요한 정보

## Memory 구동 방식
1. 메모리는 기본적으로 모든 대화 내역을 LLM Input에 밀어 넣는 것.
2. 이때 대화가 길어지면 토큰수 증가 및 성능 하락이 일어남
3. 개선 방식 컨셉
    1. 요약
    2. 적정 길이에서 앞부분 자르기
    3. 정리(특정 명사들로 정리, Node-Edge 그래프 방식 등)

# `ConversationBufferMemory`

- 메시지 저장 -> 변수에서 추출 가능

In [6]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

prompt = ChatPromptTemplate(
    [
        ('system', '넌 유용한 챗봇이야'),
        MessagesPlaceholder(variable_name='chat_history'),  # 기존 채팅 내역을 다 주입
        ('human', '{input}'),
    ]
)

memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

# 메모리를 저장할 변수는 {}다. 기존에 대화내용이 있다면 불러와라
memory.load_memory_variables({})

{'chat_history': []}

In [ ]:
from operator import itemgetter
# 'chat_history' 변수에, load_memory_var 결과를 저장하고, 'chat_history' 키를 추출
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) |
    itemgetter('chat_history')
)

runnable.invoke({'input': 'hi'})

{'input': 'hi', 'chat_history': []}

In [11]:
chain = runnable | prompt | llm

In [12]:
res = chain.invoke({'input': '만나서 반가워'})

In [17]:
memory.save_context(
    {'human': '만나서 반가워'},
    {'ai': res.content}
)

In [20]:
my_message = '내 이름은 춘식이야.'
res = chain.invoke({'input': my_message})
memory.save_context(
    {'human': my_message},
    {'ai': res.content}
)

In [22]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='만나서 반가워', additional_kwargs={}, response_metadata={}),
  AIMessage(content='저도 만나서 반가워요! 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='만나서 반가워', additional_kwargs={}, response_metadata={}),
  AIMessage(content='저도 만나서 반가워요! 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='내 이름은 춘식이야.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='반가워요, 춘식이! 어떤 이야기를 나누고 싶나요?', additional_kwargs={}, response_metadata={})]}

In [2]:
import time
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

prompt = ChatPromptTemplate(
    [
        ('system', '넌 유용한 챗봇이야'),
        MessagesPlaceholder(variable_name='chat_history'),
        ('human', '{input}'),
    ]
)

memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

# `chat_history`변수에, load_memory_var 결과를 저장하고, `chat_history 키를 추출`
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) |
    itemgetter('chat_history')
)

chain = runnable | prompt | llm 

# 아래에 실제 gpt랑 대화하는 모습으로 만들기
input_msg = ''

# 사용자가 ('quit', '정지', '그만') 중에 하나를 입력하면 대화 종료
while input_msg not in ('quit', '정지', '그만'):
    input_msg = input()
    
    # 종료 조건
    if input_msg in ('quit', '정지', '그만'):
        print("대화 종료")
        break
    
    # 대화 로직 + 대화내역 아래 출력
    print('인간: ', input_msg)
    
    # LLM 모델에 사용자 입력 전달 및 응답 받기
    response = chain.invoke({'input': input_msg})
    
    # 현재 대화 내용을 메모리에 저장
    memory.save_context({'input': input_msg}, {'output': response.content})
    
    # AI 응답 출력
    print('AI: ', response.content)
    time.sleep(0.5)

인간:  안녕 넌 누구야 ?
AI:  안녕하세요! 저는 여러분의 질문에 답변하고 도움을 드리기 위해 만들어진 AI 챗봇입니다. 무엇이든 궁금한 점이 있으면 말씀해 주세요!
인간:  난 지금 너무 잠와 어뜨케
AI:  잠이 오실 때는 몸과 마음을 편안하게 만들어주는 방법들이 도움이 될 수 있어요. 몇 가지 추천드릴게요:

1. **편안한 환경 만들기:** 조명을 낮추고 조용한 곳에서 휴식을 취하세요.
2. **이완 호흡 연습:** 깊게 숨을 들이쉬고 천천히 내쉬는 호흡을 반복하세요.
3. **가벼운 스트레칭:** 몸을 부드럽게 풀어주는 간단한 스트레칭도 도움이 될 수 있어요.
4. **전자기기 사용 줄이기:** 스마트폰이나 컴퓨터 사용을 피하고, 차분한 음악이나 자연 소리를 들어보세요.
5. **따뜻한 음료:** 우유나 허브차 같은 따뜻한 음료를 마시는 것도 도움이 될 수 있어요.

혹시 더 도움이 필요하거나, 잠이 잘 오지 않는 이유가 있다면 알려 주세요!
인간:  지금 너랑 전자기기로 대화하고 있잖아
AI:  맞아요, 지금은 전자기기를 통해 대화하고 있지만, 잠이 오기 전에 마음을 차분하게 하고 긴장을 푸는 것이 중요해요. 만약 바로 잠들기 어렵다면, 전자기기 사용을 잠시 줄이고 눈을 감거나 깊은 호흡을 하는 것도 도움이 될 수 있어요. 또는, 전자기기 사용 후에 눈을 감고 조용히 휴식을 취하는 것도 좋은 방법입니다. 더 편안한 잠을 잘 수 있도록 도와줄 수 있는 다른 방법이 필요하시면 언제든 말씀해 주세요!
인간:  아니 지금 혈당 스파이크 때문에 잠이 겁나 오는 것 같아
AI:  혈당 스파이크로 인해 졸림이나 피로를 느끼는 경우가 있을 수 있어요. 혈당이 급격히 오르면 인슐린이 많이 분비되고, 이후 혈당이 급격히 떨어지면서 졸림이나 피로를 유발할 수 있거든요. 

이럴 때는 다음과 같은 방법들이 도움이 될 수 있어요:

1. **가벼운 운동:** 가볍게 스트레칭하거나 산책을 하면 혈당 조절에 도움이 될 수 있어요.
2. **작은 식사 또는 간식:** 혈당